In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# डमी डेटा बनाना
np.random.seed(42)
num_samples = 2000

data = {
    'sq_feet': np.random.randint(500, 1000, num_samples),
    'num_bedrooms': np.random.randint(1, 4, num_samples),
    'num_bathrooms': np.random.randint(1, 3, num_samples),
    'year_built': np.random.randint(2000, 2023, num_samples),
    'location_score': np.random.rand(num_samples) * 10
}

df = pd.DataFrame(data)

# हाउस प्राइस का अनुमान लगाना (डमी)
df['price'] = (df['sq_feet'] * 150 +
               df['num_bedrooms'] * 20000 +
               df['num_bathrooms'] * 15000 +
               (df['year_built'] - 1990) * 500 +
               df['location_score'] * 10000 +
               np.random.normal(0, 10000, num_samples))

# Pandas DataFrame को PySpark DataFrame में बदलना
spark_df = spark.createDataFrame(df)

# PySpark DataFrame का स्कीमा देखें
spark_df.printSchema()

In [ ]:
# डेटा को एक अस्थायी (temporary) टेबल के रूप में रजिस्टर करें
spark_df.createOrReplaceTempView("temp_house_data")

# SQL कमांड का उपयोग करके इस अस्थायी टेबल से एक डेल्टा टेबल बनाएं
# CREATE OR REPLACE TABLE कमांड यह सुनिश्चित करता है कि अगर टेबल पहले से मौजूद है, तो वह अपडेट हो जाए
spark.sql("CREATE OR REPLACE TABLE house_price_delta USING DELTA AS SELECT * FROM temp_house_data")

print("डमी डेटा सफलतापूर्वक डेल्टा टेबल 'house_price_delta' के रूप में सेव किया गया है।")

In [ ]:
# डेल्टा टेबल से डेटा पढ़ें
read_df = spark.read.format("delta").table("house_price_delta")

# डेटा का कुछ हिस्सा दिखाएं
read_df.show(5)

# टेबल में कुल पंक्तियों (rows) की संख्या देखें
print(f"कुल पंक्तियाँ: {read_df.count()}")